In [1]:
from evaluation import JSONParseEvaluator

import os, glob, json
import argparse
import numpy as np
from tqdm import tqdm
from os.path import join, basename, splitext

In [2]:
field_list = [ 'current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt', 'diagnosis', 'date_in', 'doctor_name', 'drug_name', 'drug_dose', 'drug_quantity']

def convert_dict_concern_fileds(raw_dict: dict, concern_fileds=None, is_label=True):
    res = {}
    if concern_fileds is None:
        concern_fileds = list(raw_dict.keys())

    for field,data in raw_dict.items():
        field = field.replace(' ', '_')
        if field in concern_fileds:
            res[field] = data['value'].lower() if is_label else data.lower()
    return res


In [3]:
def convert_output(output):
    list = ['current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt']
    list_medical = ['diagnosis', 'date_in', 'doctor_name']
    list_drug = ['drug_name', 'drug_dose', 'drug_quantity']
    
    result = {}
    
    for field in list:
        try:
            result[field] = output[field]
        except:
            result[field] = ''

    if 'medical' in output.keys():
        for field in list_medical:
            try:
                result[field] = output['medical'][field]
            except:
                result[field] = ''
    if 'drugs' in output.keys():
        for field in list_drug:
            value = ""
            for drug in output['drugs']:
                for key, val in drug.items():
                    if key == field:
                        if val != "":
                            value += (val + " ")
            result[field] = value[:-1]
    return result

In [18]:
# path = "/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct_advance/Mistral-7B/Long_Chau_488.json"
# with open(path, 'r') as f:
#     pred = json.load(f)
# #convert
# print(pred)
# pred = convert_output(pred)
# print(pred)


In [4]:

path_mistral_1 = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct_advance_swap/Mistral-7B'
path_mistral_json_1 = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct_advance_swap/Mistral-7B-json'

path_labels = '/home/vinbig/Documents/PA_Modeling/Prompt/prescription_label_text/KIEs'

In [5]:
def get_outputs_labels(path_output, path_labels):
    names = os.listdir(path_output)
    name_txts = []
    preds = []
    labels = []
    for name in names:
        # check file json
        if name.split('.')[1] == 'txt':
            name_txts.append(name)
            continue

        # correct -> read output and label
        pred_path = os.path.join(path_output, name)
        with open(pred_path, 'r') as f:
            pred = json.load(f)
        #convert
        pred = convert_output(pred)

        label_path = os.path.join(path_labels, name)
        with open(label_path, 'r') as f:
            label = json.load(f)
        
        label = convert_dict_concern_fileds(label, concern_fileds=field_list)
        pred  = convert_dict_concern_fileds(pred, concern_fileds=field_list, is_label=False)

        print("predic:", pred)
        print("label:", label)
        print('-----------')
        labels.append(label)
        preds.append(pred)
    return preds, labels, name_txts

In [6]:
def compute_acc(preds, labels):
    evaluator = JSONParseEvaluator()
    accs = []
    for i in range (len(labels)):
        acc = evaluator.cal_acc(pred=preds[i], answer=labels[i])
        accs.append(acc)
    return np.mean(accs)

In [7]:
def get_each_field(field, input):
    result = {}
    try:
        result[field] = input[field]
    except:
        result[field] = ""
    return result

In [8]:
def compute(preds, labels, name_txts, field_list):
    print("Số lượng file đúng format: ", len(preds), '/249')
    acc_mean = []
    # all field
    acc = compute_acc(preds, labels)
    acc_mean.append(acc)

    #each field
    for field in field_list:
        preds_field = [get_each_field(field=field, input=pred) for pred in preds]
        labels_field = [get_each_field(field=field, input=label) for label in labels]
        acc_mean.append(compute_acc(preds_field, labels_field))

    return acc_mean

In [9]:
print('Mistral')
preds, labels, name_txts = get_outputs_labels(path_labels=path_labels, path_output=path_mistral_1)
acc_mean = compute(preds, labels, name_txts, field_list)

Mistral
predic: {'current_institute': 'sởy tế tp. hồ chí minh bệnh viện nhi đồng 2 7 9 4 0 8 2 2 0 0 9 2 6 7 5 ', 'name': 'kheang rama pannhan', 'gender': 'nam', 'birth': '2t10th', 'age': '2t10th', 'address': '', 'tel_customer': '0368688847', 'id_bhyt': '', 'diagnosis': 'r63.0-chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'date_in': '', 'doctor_name': '', 'drug_name': 'sắt (iii) hydroxyo polymaltose (amkuk 50mg/ 5ml- 100ml (chai))- 50mg/ 5ml', 'drug_dose': 'buổi: trưa', 'drug_quantity': '02 chai'}
label: {'current_institute': 'bệnh viện nhi đồng 2', 'name': 'kheang rama pannhan', 'gender': 'nam', 'age': '2 t 10 th', 'address': '', 'tel_customer': '0368688847', 'diagnosis': 'r63.0 - chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'drug_quantity': '02 chai', 'drug_name': 'sắt (iii) hydroxyd polymaltose (amkuk 50mg/ 5ml - 100ml (chai)) - 50mg/ 5ml', 'drug_dose': 'uống mỗi ngày 1 lần, mỗi lần 2.5 ml buổi: trưa'}
-----------
predic: {'current_institute'

/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: invalid value encountered in scalar divide
  zss.distance(
/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: divide by zero encountered in scalar divide
  zss.distance(


In [10]:
print('Mistral Json')
preds_json, labels_json, name_txts_json = get_outputs_labels(path_labels=path_labels, path_output=path_mistral_json_1)
acc_mean_json = compute(preds_json, labels_json, name_txts_json, field_list)

Mistral Json
predic: {'current_institute': 'sởy tế tp. hồ chí minh bệnh viện nhi đồng 2 7 9 4 0 8 2 2 0 0 9 2 6 7 5 ', 'name': 'kheang rama pannhan', 'gender': 'nam', 'birth': '2t10th', 'age': '2t10th', 'address': 'cha mẹ/người giám hộ: phan kim nga - sđt: 0368688847', 'tel_customer': '0368688847', 'id_bhyt': '', 'diagnosis': 'r63.0-chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'date_in': '', 'doctor_name': '', 'drug_name': 'sắt (iii) hydroxyo polymaltose (amkuk 50mg/ 5ml- 100ml (chai))', 'drug_dose': 'buổi: trưa', 'drug_quantity': '02 chai'}
label: {'current_institute': 'bệnh viện nhi đồng 2', 'name': 'kheang rama pannhan', 'gender': 'nam', 'age': '2 t 10 th', 'address': '', 'tel_customer': '0368688847', 'diagnosis': 'r63.0 - chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'drug_quantity': '02 chai', 'drug_name': 'sắt (iii) hydroxyd polymaltose (amkuk 50mg/ 5ml - 100ml (chai)) - 50mg/ 5ml', 'drug_dose': 'uống mỗi ngày 1 lần, mỗi lần 2.5 ml buổi: t

In [11]:
import pandas as pd
model_name = ["mistral", "mistral_json"]
column_name_all = ['all', 'current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt', 'diagnosis', 'date_in', 'doctor_name', 'drug_name', 'drug_dose', 'drug_quantity']
df = pd.DataFrame([acc_mean, acc_mean_json], columns=column_name_all, index=model_name)
df

,all,current_institute,name,gender,birth,age,address,tel_customer,id_bhyt,diagnosis,date_in,doctor_name,drug_name,drug_dose,drug_quantity
mistral,0.723374,0.551062,0.949632,0.793902,0.459754,0.209940,0.859232,0.401075,0.295331,0.852623,0.686076,0.536598,0.740458,0.662489,0.758336
mistral_json,0.724039,0.552728,0.949797,0.788980,0.453468,0.212611,0.854787,0.398630,0.296536,0.851450,0.689819,0.535185,0.741840,0.666978,0.763357


In [13]:

path_mistral = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct_advance/Mistral-7B'
path_mistral_json = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct_advance/Mistral-7B-json'


In [14]:
print('Mistral')
preds_0, labels_0, name_txts_0 = get_outputs_labels(path_labels=path_labels, path_output=path_mistral)
acc_mean_0 = compute(preds_0, labels_0, name_txts_0, field_list)

Mistral
predic: {'current_institute': 'sởy tế tp. hồ chí minh bộ môi trường', 'name': 'kheang rama pannhan', 'gender': 'nam', 'birth': '2t10th', 'age': '', 'address': '', 'tel_customer': '0368688847', 'id_bhyt': '', 'diagnosis': 'r63.0-chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'date_in': '', 'doctor_name': '', 'drug_name': 'sắt (iii) hydroxyo polymaltose (amkuk 50mg/ 5ml- 100ml (chai))', 'drug_dose': 'buổi: trưa', 'drug_quantity': '02 chai'}
label: {'current_institute': 'bệnh viện nhi đồng 2', 'name': 'kheang rama pannhan', 'gender': 'nam', 'age': '2 t 10 th', 'address': '', 'tel_customer': '0368688847', 'diagnosis': 'r63.0 - chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'drug_quantity': '02 chai', 'drug_name': 'sắt (iii) hydroxyd polymaltose (amkuk 50mg/ 5ml - 100ml (chai)) - 50mg/ 5ml', 'drug_dose': 'uống mỗi ngày 1 lần, mỗi lần 2.5 ml buổi: trưa'}
-----------
predic: {'current_institute': 'bệnh viện da liễu', 'name': 'nguyễn thị dung', 'ge

/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: invalid value encountered in scalar divide
  zss.distance(
/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: divide by zero encountered in scalar divide
  zss.distance(


In [15]:
print('Mistral Json')
preds_json_0, labels_json_0, name_txts_json_0 = get_outputs_labels(path_labels=path_labels, path_output=path_mistral_json)
acc_mean_json_0 = compute(preds_json_0, labels_json_0, name_txts_json_0, field_list)

Mistral Json
predic: {'current_institute': 'sởy tế tp. hồ chí minh bộ môi trường', 'name': 'kheang rama pannhan', 'gender': 'nam', 'birth': '2t10th', 'age': '', 'address': '', 'tel_customer': '0368688847', 'id_bhyt': '', 'diagnosis': 'r63.0-chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'date_in': '', 'doctor_name': '', 'drug_name': 'sắt (iii) hydroxyo polymaltose (amkuk 50mg/ 5ml- 100ml (chai))', 'drug_dose': 'buổi: trưa', 'drug_quantity': '02 chai'}
label: {'current_institute': 'bệnh viện nhi đồng 2', 'name': 'kheang rama pannhan', 'gender': 'nam', 'age': '2 t 10 th', 'address': '', 'tel_customer': '0368688847', 'diagnosis': 'r63.0 - chán ăn ;dị ứng đậu nành, lòng trắng trứng/ thiếu máu thiếu sắt', 'drug_quantity': '02 chai', 'drug_name': 'sắt (iii) hydroxyd polymaltose (amkuk 50mg/ 5ml - 100ml (chai)) - 50mg/ 5ml', 'drug_dose': 'uống mỗi ngày 1 lần, mỗi lần 2.5 ml buổi: trưa'}
-----------
predic: {'current_institute': 'bệnh viện da liễu', 'name': 'nguyễn thị dung'

In [16]:
model_name = ["mistral", "mistral_json"]
column_name_all = ['all', 'current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt', 'diagnosis', 'date_in', 'doctor_name', 'drug_name', 'drug_dose', 'drug_quantity']
df_0 = pd.DataFrame([acc_mean_0, acc_mean_json_0], columns=column_name_all, index=model_name)
df_0

,all,current_institute,name,gender,birth,age,address,tel_customer,id_bhyt,diagnosis,date_in,doctor_name,drug_name,drug_dose,drug_quantity
mistral,0.733200,0.574022,0.950088,0.785998,0.447559,0.197671,0.861992,0.392319,0.287709,0.840063,0.691056,0.558688,0.746597,0.663451,0.764970
mistral_json,0.734444,0.567895,0.952720,0.789474,0.449796,0.204158,0.864862,0.399196,0.290340,0.835733,0.691077,0.560385,0.746444,0.665932,0.767178


In [13]:
import json
path = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct/Mistral-7B/Long_Chau_509.json'
with open (path, 'r') as f:
    out = json.load(f)
out

{'patient': {'name': 'TA QUANG MINH (MR.)',
  'gender': '',
  'birth': '17 Oct 1985',
  'age': '37Y',
  'address': 'HN: 218/3 An Dương Vương, phường 16, Quận 8, Thành phố Hồ Chim',
  'tel_customer': '',
  'id_bhyt': ''},
 'medical': {'current_institute': 'BỆNH VIỆN Guardian:',
  'diagnosis': '[K74] FIBROSIS AND CIRRHOSIS OF LIVER',
  'date_in': '12 Dec 2022',
  'doctor_name': 'Deputy Head of Gastro-nntrrollyy and Hepatology'},
 'drugs': [{'drug_name': 'Mirtazapine (REMERON) tab 30mg',
   'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày vào buổi sáng trong 30 ngày',
   'drug_quantity': '08 Viên'},
  {'drug_name': 'Propranolol (PROPRANOLOL TV PHARM) tab 40mg',
   'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày lúc đi ngủ trong 30 ngày',
   'drug_quantity': '08 Viên'},
  {'drug_name': 'Thiamine (Vit B1) (VITAMIN B1) tab 250mg',
   'drug_dose': 'Mỗi lần uống 1 viên hai lần một ngày sau khi ăn trong 30 ngày',
   'drug_quantity': '60 Viên'},
  {'drug_name': '(LIVACT) 4.15 g/sachet

In [14]:
convert_output(out)

{'current_institute': '',
 'name': '',
 'gender': '',
 'birth': '',
 'age': '',
 'address': '',
 'tel_customer': '',
 'id_bhyt': '',
 'diagnosis': '[K74] FIBROSIS AND CIRRHOSIS OF LIVER',
 'date_in': '12 Dec 2022',
 'doctor_name': 'Deputy Head of Gastro-nntrrollyy and Hepatology',
 'drug_name': 'Mirtazapine (REMERON) tab 30mgPropranolol (PROPRANOLOL TV PHARM) tab 40mgThiamine (Vit B1) (VITAMIN B1) tab 250mg(LIVACT) 4.15 g/sachet oral granulesUrsodeoxycholic acid (URSOBIL 300) tab 300mg',
 'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày vào buổi sáng trong 30 ngàyMỗi lần uống 0.25 viên một lần mỗi ngày lúc đi ngủ trong 30 ngàyMỗi lần uống 1 viên hai lần một ngày sau khi ăn trong 30 ngàyMỗi lần uống ba lần một ngày sau khi ăn trong 30 ngàyMỗi lần uống 1 viên ba lần một ngày sau khi ăn trong 30 ngày',
 'drug_quantity': '08 Viên08 Viên60 Viên90 Viên90 Viên'}